In [1]:
# this work relies on https://github.com/schmrlng/MotionPlanning.jl, please download this first

using PyPlot
using MotionPlanning
include(Pkg.dir("MotionPlanning")*"/test/obstaclesets/2D.jl")
include(Pkg.dir("MotionPlanning")*"/test/obstaclesets/ND.jl")

INFO: Loading help data...


In [8]:
# Generate n by d Halton samples
function haltonsamples(d, n)
    bases = primes(100);
    samples = zeros(n, d);
    for i = 1:d
        b = bases[i];
        samples[:,i] = haltonsequence(b, n)
    end
    return samples
end;


# Generate the first n numbers in Halton's sequence with base b.
#
#   seq = haltonsequence(b, n)
#
# Output:
#  seq - sequence of n Halton numbers
#
# Inputs:
#  b - base of the sequence
#  n - length of vector to be generated
#
function haltonsequence(b, n)
  map((i) -> haltonnumber(b, i), 1:n)
end;

# Generate the nth Halton number in the sequence with base b.
#
# Notes:
#  Implementation is based on the psudo code in:
#    http://en.wikipedia.org/wiki/Halton_sequence
#

function haltonnumber(base, index)
  res = 0
  f = 1 / base
  i = index

  while (i > 0)
    res += f * (i % base)
    i = ifloor(i / base)
    f = f / base
  end

  return res
end;

function RSgridsamples(n)
    Nx = convert(Int64,ceil(n^(1/4)))
    Ny = Nx
    Nt = convert(Int64,ceil(sqrt(n/(Nx*Ny))))
    xrange = linspace(1/(2*Nx), 1 - 1/(2*Nx),Nx)
    yrange = linspace(1/(2*Ny), 1 - 1/(2*Ny),Ny)
    dx = xrange[2] - xrange[1]
    dy = yrange[2] - yrange[1]
    trange = linspace(0,2pi,Nt^2+1)[1:end-1]
    samples = ([(xy = ([x,y] + [ind2sub((Nt,Nt),t)...].*[dx,dy]/Nt); (xy[1], xy[2], trange[t]))
        for x in xrange,
        y in yrange,
        t in 1:Nt^2]) |> vec
end;

function DIgridsamples(n, vmax)
    Nx = convert(Int64,ceil(n^(1/4)))
    Ny = Nx
    Nv = convert(Int64,ceil((n/Nx^2)^(1/2)))
    xrange = linspace(1/(2*Nx), 1 - 1/(2*Nx),Nx)
    yrange = linspace(1/(2*Ny), 1 - 1/(2*Ny),Ny)
    dx = xrange[2] - xrange[1]
    dy = yrange[2] - yrange[1]
    
    vrange = linspace(-1 + 1/Nv, 1 - 1/Nv,Nv)
    
    samples = [(xy = ([x,y] + [vx,vy].*[dx,dy]/2); [xy[1], xy[2], 
            vx*vmax, vy*vmax])
        for x in xrange,
        y in yrange,
        vx in vrange,
        vy in vrange] |> vec
end;

function DIpertgridsamples(n, vmax)
    Nx = convert(Int64,ceil(n^(1/4)))
    Ny = Nx
    Nv = convert(Int64,ceil((n/Nx^2)^(1/2)))
       
    Ntmp = Nx
    Nx = Nv
    Ny = Nv
    Nv = Ntmp
    
    xrange = linspace(1/(2*Nx), 1 - 1/(2*Nx),Nx)
    yrange = linspace(1/(2*Ny), 1 - 1/(2*Ny),Ny)
    dx = xrange[2] - xrange[1]
    dy = yrange[2] - yrange[1]
    
    vrange = linspace(-1 + 1/Nv, 1 - 1/Nv,Nv)
    dv = vrange[2] - vrange[1]
    
    offpos = 0.001 # error in the 2pt bvp implementation, add a little randomness
    offvel = 0.03 # error in the 2pt bvp implementation, add a little randomness
    idx = 0;
    
    # flipped perm
    samples = [(idx = idx + 1; odd = idx % 2; odd == 0 ? flip = 1 : flip = -1; 
        Nv % 2 == 0 && idx % Nv == 0 ? idx = idx + 1 : nothing;
        xy = ([x,y] + [vx,vy].*[dx,dy]/2); 
        [xy[1]+rand(1)*offpos-offpos/2, xy[2]+rand(1)*offpos-offpos/2, 
            flip*vperm[findfirst(vrange,vx)]*vmax+rand(1)*offvel-offvel/2, 
            flip*vperm[findfirst(vrange,vy)]*vmax+rand(1)*offvel-offvel/2])
            for  vx in vrange, 
        vy in vrange,
        x in xrange,
        y in yrange] |> vec
    
end;

In [58]:
# define initial reeds shepp problem
SS = ReedsSheppStateSpace(.2)   # ReedsSheppStateSpace, dist = arc length
init = RSState(.1,.1,0.)
goal = PointGoal([.9,.9])
CC = PointRobot2D(ISRR_POLY_WITH_SPIKE);

In [60]:
# lattice based sampling
runs = 1;
N = 1000; # bug when under 80 points
D = 3
r0 = 1.6*(log(N)/N)^(1/D); # fix to correct one used
println("r is $r0")

P = MPProblem(SS, init, goal, CC);
samples = RSgridsamples(N)
N = size(samples,1)
P.V.V = map(v -> RSState(v...), [(samples[i]) for i in 1:N]) |> vec
P.V.V[end] = RSState(.9,.9,0.)
setup_steering(P.SS, r0)
P.V.V[1] = init
initcache(P.V)
prmstar!(P, N, connections = :R, r = r0), P.solution.status
if P.status != :solved
    println("Lattice failed!")
end 
# plot(P, meta=true)
latticeCost = P.solution.cost

# halton sampling implementation
P = MPProblem(SS, init, goal, CC);
samples = haltonsamples(D, N)
samplesSates = map(v -> RSState(v...), [((samples[i,1], samples[i,2], 2*pi*samples[i,3]))
    for i in 1:N]) |> vec
P.V.V = samplesSates
P.V.V[end] = RSState(.9,.9,0.)
setup_steering(P.SS, r0)
P.V.V[1] = init
initcache(P.V)
prmstar!(P, N, connections = :R, r = r0), P.solution.status
if P.status != :solved
    println("Halton failed!")
end
# plot(P, meta=true)
haltonCost = P.solution.cost
println("With N = $N, halton = $haltonCost, lattice = $latticeCost")

# random samples
results = zeros(1,runs);
numSuccess = 0;
Nfree = convert(Int64,floor(N*(1-0.32)))
for run = 1:runs
    P = MPProblem(SS, init, goal, CC);
    prmstar!(P, Nfree, connections = :R, r = r0)
    if P.status == :solved
        results[run] = P.solution.cost;
        numSuccess += 1
    else
        results[run] = NaN;
    end
    mod(run,1) == 0 ? println("$run") : 0;
end
# plot(P, meta=true)
successRate = numSuccess/runs
avgCost = mean(results[!isnan(results)])
stdCost = std(results[!isnan(results)]/sqrt(numSuccess))
println("With $Nfree points (random), \n the average cost = $avgCost +- $stdCost \n and success rate = $successRate")

r is 0.3047185996224888
With N = 1296, halton = 1.4710227063796013, lattice = 1.4854481472122494
1
With 881 points (random), 
 the average cost = 1.3857166777964074 +- NaN 
 and success rate = 1.0


In [2]:
# Geometric
SS = UnitHypercube(2)   # RealVectorMetricSpace, dist = Euclidean
start = [.1, .1]
goal = PointGoal([.9, .9])
CC = PointRobot2D(ISRR_POLY_WITH_SPIKE)
P = MPProblem(SS, start, goal, CC);

In [7]:
fmtstar!(P, 4000, connections = :R, rm = 1.5)
adaptive_shortcut!(P)
plot(P, meta=true, smoothed=true)

LoadError: Adaptive-shortcut requires Euclidean SS
while loading In[7], in expression starting on line 2

In [3]:
# DOUBLE INTEGRATOR
vmax0 = 0.5;
runs = 50;
N = 3000; # bug when under 80 points
D = 4
statespace = DoubleIntegrator(2, vmax = vmax0)   # LinearQuadraticStateSpace, dist = time + quadratic control cost
init = [.1, .1, 0., 0.]
goal = PointGoal([.9, .9, 0., 0.])
collisionchecker = PointRobot2D(ISRR_POLY_WITH_SPIKE);
r0 = 6*(log(N)/N)^(1/D);
println("r is $r0")

samples = DIpertgridsamples(N, vmax0) # DIpertgridsamples(N, vmax0)
N = size(samples,1)

# LATTICE SAMPLING
resultsLattice = zeros(1,runs);
numSuccessLattice = 0;
P = MPProblem(statespace, init, goal, collisionchecker);
for run = 1:runs
#     P = MPProblem(statespace, init, goal, collisionchecker);
    samples = DIpertgridsamples(N, vmax0) # DIpertgridsamples(N, vmax0)
    N = size(samples,1)
    P.V.V = [samples[i] for i in 1:N] |> vec
    P.V.V[end] = [.9, .9, 0., 0.]
    P.V.V[1] = init
    setup_steering(P.SS, r0)
    initcache(P.V)
    fmtstar!(P, N, connections = :R, r = r0), P.solution.status
    if P.status == :solved
        resultsLattice[run] = P.solution.cost;
        numSuccessLattice += 1
    else
        resultsLattice[run] = NaN;
    end
    mod(run,1) == 0 ? println("$run") : 0;
end
# plot(P, meta=true)
successLattice = numSuccessLattice/runs
latticeStdCost = std(resultsLattice[!isnan(resultsLattice)]/sqrt(numSuccessLattice))
latticeCost = mean(resultsLattice[!isnan(resultsLattice)])
println("With N = $N, success rate = $successLattice, lattice = $latticeCost +- $latticeStdCost")

latticeCost = 0;

# HALTON SAMPLING
# adjust number of sampels
samples = DIpertgridsamples(N, vmax0) # DIpertgridsamples(N, vmax0)
N = size(samples,1)
# setup problem and run with halton
P = MPProblem(statespace, init, goal, collisionchecker);
samples = haltonsamples(4, N)
samples[:,3] = samples[:,3]*2*vmax0-vmax0
samples[:,4] = samples[:,4]*2*vmax0-vmax0
P.V.V = [samples[i,:] |> vec for i in 1:N] |> vec
P.V.V[end] = [.9, .9, 0., 0.]
P.V.V[1] = init
setup_steering(P.SS, r0)
initcache(P.V)
fmtstar!(P, N, connections = :R, r = r0), P.solution.status
if P.status != :solved
    println("Halton failed!")
end
# plot(P, meta=true)
haltonCost = P.solution.cost
println("With N = $N, halton = $haltonCost, lattice = $latticeCost")

# RANDOM SAMPLING
results = zeros(1,runs);
numSuccess = 0;
Nfree = convert(Int64,floor(N*(1-0.32)))
for run = 1:runs
    P = MPProblem(statespace, init, goal, collisionchecker);
    prmstar!(P, Nfree, connections = :R, r = r0)
    if P.status == :solved
        results[run] = P.solution.cost;
        numSuccess += 1
    else
        results[run] = NaN;
    end
    mod(run,5) == 0 ? println("$run") : 0;
end
plot(P, meta=true)
successRate = numSuccess/runs
avgCost = mean(results[!isnan(results)])
stdCost = std(results[!isnan(results)]/sqrt(numSuccess))
println("With $Nfree points (random), \n the average cost = $avgCost +- $stdCost \n and success rate = $successRate")

r is 1.3637344695808702
1


LoadError: interrupt
while loading In[3], in expression starting on line 20